In [114]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [115]:
dataset = pd.read_excel('../data/boxScores.xlsx')
# Filter out rows where the 'MIN' column is zero
dataset = dataset[dataset['MIN'] != 0]
dataset.sort_values(by=['GAME DATE'], ascending = [False], inplace=True)


In [116]:
dataset.describe()

,GAME DATE,MIN,PTS,FGM,FGA,3PM,3PA,FTM,FTA,OREB,...,PF,+/-,FP,Last3_FP_Avg,Last3_Min,Last5_FP_Avg,Last5_Min,Last7_FP_Avg,Last7_Min,Season_FP_Avg
count,9982,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,...,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000
mean,2024-11-22 14:11:51.200160256,22.482068,10.529253,3.836806,8.263474,1.261170,3.502104,1.594470,2.046584,1.012222,...,1.785514,-0.000701,21.593288,21.480266,22.362369,21.392051,22.268400,21.322921,22.201792,21.574098
min,2024-10-22 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-43.000000,-3.000000,-1.000000,0.666667,-1.000000,1.000000,-1.000000,1.000000,-1.000000
25%,2024-11-06 00:00:00,14.000000,3.000000,1.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,-7.000000,9.700000,11.437500,14.666667,11.661667,14.800000,11.728571,14.723214,12.375000
50%,2024-11-21 12:00:00,24.000000,9.000000,3.000000,7.000000,1.000000,3.000000,1.000000,1.000000,1.000000,...,2.000000,0.000000,19.800000,20.066667,23.333333,20.000000,23.200000,19.900000,23.285714,20.084375
75%,2024-12-08 00:00:00,31.000000,16.000000,6.000000,12.000000,2.000000,5.000000,2.000000,3.000000,2.000000,...,3.000000,7.000000,31.200000,29.891667,30.666667,29.675000,30.600000,29.480357,30.428571,28.750000
max,2024-12-28 00:00:00,48.000000,60.000000,22.000000,39.000000,10.000000,20.000000,17.000000,26.000000,12.000000,...,6.000000,43.000000,98.300000,80.500000,44.666667,75.360000,44.000000,72.200000,44.000000,63.355556
std,NaN,10.847488,8.817063,3.234562,6.034359,1.537369,3.171918,2.246381,2.715058,1.343533,...,1.452097,11.310923,14.981087,13.138480,10.047682,12.732902,9.865021,12.570748,9.792995,11.903855


In [117]:
# Filter out players who aren't in clusters for accuracy
clusterdf = pd.read_excel('../data/clusteredPlayers.xlsx')
clusterdf = clusterdf.drop('Unnamed: 0', axis=1)
clusterDict = {}
clusterDict.update(pd.Series(clusterdf['Cluster'].values, index=clusterdf['Player']).to_dict())
dataset['CLUSTER'] = dataset['PLAYER'].map(clusterDict).astype('category')
dataset = dataset.dropna(subset=['CLUSTER'])
len(dataset)

7538

In [118]:
dataset.describe()

,GAME DATE,MIN,PTS,FGM,FGA,3PM,3PA,FTM,FTA,OREB,...,PF,+/-,FP,Last3_FP_Avg,Last3_Min,Last5_FP_Avg,Last5_Min,Last7_FP_Avg,Last7_Min,Season_FP_Avg
count,7538,7538.000000,7538.000000,7538.000000,7538.000000,7538.000000,7538.000000,7538.000000,7538.000000,7538.000000,...,7538.000000,7538.000000,7538.000000,7538.000000,7538.000000,7538.000000,7538.000000,7538.000000,7538.000000,7538.000000
mean,2024-11-22 12:56:44.192093440,26.475458,12.720483,4.628283,9.919209,1.529185,4.205625,1.934731,2.462192,1.153622,...,2.046431,0.195145,25.763956,25.650294,26.363956,25.571644,26.282005,25.510521,26.222466,25.757884
min,2024-10-22 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-43.000000,-2.800000,-1.000000,1.000000,-1.000000,1.000000,-1.000000,1.000000,8.088889
25%,2024-11-06 00:00:00,20.000000,6.000000,2.000000,6.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,1.000000,-8.000000,15.200000,16.933333,21.000000,17.100000,21.000000,17.189286,20.857143,17.867857
50%,2024-11-21 00:00:00,27.000000,11.000000,4.000000,9.000000,1.000000,4.000000,1.000000,2.000000,1.000000,...,2.000000,0.000000,24.400000,24.200000,26.666667,24.163333,26.600000,24.100000,26.666667,23.887500
75%,2024-12-08 00:00:00,33.000000,18.000000,7.000000,13.000000,2.000000,6.000000,3.000000,4.000000,2.000000,...,3.000000,8.000000,34.575000,33.000000,32.666667,32.560000,32.333333,32.435714,32.191667,32.138462
max,2024-12-28 00:00:00,48.000000,60.000000,22.000000,39.000000,10.000000,20.000000,17.000000,26.000000,12.000000,...,6.000000,43.000000,98.300000,80.500000,44.666667,75.360000,44.000000,72.200000,44.000000,63.355556
std,NaN,8.552300,8.730300,3.208314,5.793261,1.618032,3.222691,2.388939,2.871715,1.414594,...,1.433306,12.305830,14.161038,11.917093,7.522340,11.403899,7.279630,11.191780,7.176671,10.321330


For reference, features are the raw input data provided to the model to generate predictions, while the label is the target outcome that the model aims to predict. These above features can and probably should be tweaked and adjusted to improve the model's performance.

In [119]:
featureNames = ['Last3_FP_Avg', 'Last5_FP_Avg', 'Last7_FP_Avg', 'Season_FP_Avg']
labelName = ['FP']
dfFeatures = dataset[featureNames]
dfFeatures.head()

,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg
6550,14.2,20.14,19.357143,18.230769
3620,11.3,9.58,8.714286,10.827273
5411,14.8,12.28,15.314286,15.665517
992,13.8,18.70,20.214286,20.766667
6031,45.7,47.18,47.542857,43.704762


In [120]:
dfLabels = dataset[labelName]
dfLabels.head()

,FP
6550,18.1
3620,6.6
5411,20.9
992,2.4
6031,46.7


In [121]:
labels = np.array(dfLabels)
features = np.array(dfFeatures)

** Note numpy array is only numbers while pandas dataframe can be a mix of numbers and strings.

In [122]:
# Train is your training data while test is your testing data
train, test, trainLabels, testLabels = train_test_split(features, labels, test_size=0.2, random_state=30)
# In general a test_size should be small, always less than 50%


In [123]:
# Scaling the data
x = features
x = StandardScaler().fit_transform(x)

In [124]:
train, test, trainLabels, testLabels = train_test_split(x, labels, test_size=0.2, random_state=30)

In [125]:
dataset = dataset[['PLAYER', 'FP', 'Last3_FP_Avg', 'Last5_FP_Avg', 'Last7_FP_Avg', 'Season_FP_Avg']]
players = dataset['PLAYER']

In [126]:
featureNames = ['Last3_FP_Avg', 'Last5_FP_Avg', 'Last7_FP_Avg', 'Season_FP_Avg']
labelName = ['FP']
dfFeatures = dataset[featureNames]
dfFeatures.head()

,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg
6550,14.2,20.14,19.357143,18.230769
3620,11.3,9.58,8.714286,10.827273
5411,14.8,12.28,15.314286,15.665517
992,13.8,18.70,20.214286,20.766667
6031,45.7,47.18,47.542857,43.704762


In [127]:
dfLabels = dataset[labelName]
dfLabels.head()

,FP
6550,18.1
3620,6.6
5411,20.9
992,2.4
6031,46.7


In [128]:
labels = np.array(dfLabels)
features = np.array(dfFeatures)
train, test, trainLabels, testLabels, trainPlayers, testPlayers= train_test_split(features, labels, players,  test_size=0.2, random_state=30)
testPlayers = testPlayers.reset_index(drop=True)


In [129]:
rf = RandomForestRegressor(random_state=30)

In [130]:
rf.fit(train, trainLabels)

c:\Users\Ian Bracken\Desktop\WebDev\nba-prediction-model\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=30)

In [131]:
rfPredNoStandard = rf.predict(test)

In [132]:
df3 = pd.DataFrame(test, columns = featureNames)
df3['PLAYER'] = testPlayers
df3['actual'] = testLabels
df3['predicted'] = rfPredNoStandard
df3['error'] = abs(df3['actual'] - df3['predicted'])

In [133]:
train, test, trainLabels, testLabels = train_test_split(x, labels, test_size=0.2, random_state=30)
rf = RandomForestRegressor(random_state=30)
rf.fit(train, trainLabels)
rfPredNoStandard = rf.predict(test)

c:\Users\Ian Bracken\Desktop\WebDev\nba-prediction-model\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [134]:
df4 = pd.DataFrame(test, columns = featureNames)
df4['actual'] = testLabels
df4['predicted'] = rfPredNoStandard
df4['error'] = abs(df4['actual'] - df4['predicted'])

In [135]:
df4.describe()

,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg,actual,predicted,error
count,1508.000000,1508.000000,1508.000000,1508.000000,1508.000000,1508.000000,1508.000000
mean,0.016828,0.027504,0.027945,0.032456,25.702122,26.035829,6.344442
std,1.001784,0.996852,1.006705,1.019636,14.053181,12.231745,5.178532
min,-2.236457,-2.330204,-2.368906,-1.712005,-1.000000,0.024000,0.001000
25%,-0.730117,-0.709598,-0.700480,-0.734950,15.100000,16.721500,2.220000
50%,-0.102126,-0.077316,-0.101999,-0.164877,24.350000,24.598000,5.184000
75%,0.646149,0.663160,0.648814,0.650139,34.700000,33.840250,9.374000
max,3.626660,3.455921,3.490693,3.642957,89.700000,68.084000,42.262000


In [136]:
# 3 isn't standardized which makes featurenames more readable
# Reorder columns to make PLAYER the first column
df3 = df3[['PLAYER'] + [col for col in df3.columns if col != 'PLAYER']]
df3.head()

,PLAYER,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg,actual,predicted,error
0,Bam Adebayo,35.200000,37.28,39.914286,40.072414,42.9,37.160,5.740
1,OG Anunoby,37.033333,37.72,36.928571,31.328125,39.1,38.773,0.327
2,Andrew Nembhard,29.600000,28.58,25.785714,21.964706,45.3,24.281,21.019
3,Brandin Podziemski,25.566667,29.58,27.200000,20.766667,39.4,23.642,15.758
4,Quenton Jackson,20.233333,22.70,19.342857,13.243750,28.0,22.647,5.353


In [137]:
pd.DataFrame.to_excel(df3, '../data/playerPredictions.xlsx')

In [138]:
dfCheck = pd.DataFrame()
# Difference in accuracy between df3 and df4 is negligible so I am sticking with 3 for better readability
dfCheck['df3'] = df3['error']
dfCheck['df4'] = df4['error']
dfCheck.describe()

,df3,df4
count,1508.000000,1508.000000
mean,6.348752,6.344442
std,5.180028,5.178532
min,0.001000,0.001000
25%,2.226750,2.220000
50%,5.212500,5.184000
75%,9.421000,9.374000
max,42.262000,42.262000


Note: Scaling and preprocessing data matters more on larger datasets